In [5]:
perf_filepath = "output/20230209-000554-pr.pkl"

In [114]:
import pickle
from pathlib import Path
import torch
import torch.utils.benchmark as benchmark

ab_results = []
assert Path(perf_filepath).exists(), f"{perf_filepath} is not found"
with open(perf_filepath, "rb") as handler:
    output = pickle.load(handler)
    ab_results.extend(output["test_results"])

    
compare = benchmark.Compare(ab_results)

In [115]:
# compare.colorize()
compare.

[-------------------------------------------------------------------- Resize -------------------------------------------------------------------]
                                                                 |  Pillow (9.0.0.post1)  |  torch (2.0.0a0+gite6bdca1) PR  |  torchvision resize
1 threads: --------------------------------------------------------------------------------------------------------------------------------------
      3 torch.uint8 channels_last bilinear 256 -> 32 aa=True     |          38.3          |              132.5              |         367.6      
      3 torch.uint8 channels_last bilinear 256 -> 32 aa=False    |                        |              111.2              |          73.4      
      3 torch.uint8 channels_last bilinear 520 -> 32 aa=True     |         112.0          |              440.6              |        1592.0      
      3 torch.uint8 channels_last bilinear 520 -> 32 aa=False    |                        |              362.2              

In [13]:
compare._render??

Signature: compare._render()
Docstring: <no docstring>
Source:   
    def _render(self):
        results = common.Measurement.merge(self._results)
        grouped_results = self._group_by_label(results)
        output = []
        for group in grouped_results.values():
            output.append(self._layout(group))
        return output
File:      /opt/conda/lib/python3.9/site-packages/torch/utils/benchmark/utils/compare.py
Type:      method


In [205]:
from torch.utils.benchmark.utils import common

In [206]:
results = common.Measurement.merge(compare._results)

In [207]:
grouped_results = compare._group_by_label(results)

In [213]:
c1 = "torch (2.0.0a0+gite6bdca1) PR"
c2 = "torchvision resize"
description = f"Speed-up: {c1} vs {c2}"

In [214]:
class Value(common.Measurement): pass
#     @property
#     def times(self):
#         assert len(self.raw_times) == 1, self.raw_times
#         return self.raw_times[0]
    
#     def _lazy_init(self):
#         pass

In [215]:
updated_group = []

v1 = None
v2 = None
r = None

_, scale = common.select_unit(min([r.median for r in group]))

for measurement in group:

    if measurement.task_spec.description == c1:
        v1 = measurement.median
    
    if measurement.task_spec.description == c2:
        v2 = measurement.median
    
    updated_group.append(measurement)
    if v1 is not None and v2 is not None:    
        r = v2 / v1 * scale
        v1 = None
        v2 = None
        speedup_task = common.TaskSpec(
            "", 
            setup="", 
            label=measurement.label,
            sub_label=measurement.sub_label,
            num_threads=measurement.num_threads,
            env=measurement.env,
            description=description
        )
        speedup_measurement = Value(1, [r, ], speedup_task)
        r = None
        updated_group.append(speedup_measurement)

In [216]:
from torch.utils.benchmark.utils.compare import Table


class CustomizedTable(Table):
    
    def __init__(self, results, colorize, trim_significant_figures, highlight_warnings):
        assert len(set(r.label for r in results)) == 1

        self.results = results
        self._colorize = colorize
        self._trim_significant_figures = trim_significant_figures
        self._highlight_warnings = highlight_warnings
        self.label = results[0].label
        self.time_unit, self.time_scale = common.select_unit(
            min(r.median for r in results if not isinstance(r, Value))
        )

        self.row_keys = common.ordered_unique([self.row_fn(i) for i in results])
        self.row_keys.sort(key=lambda args: args[:2])  # preserve stmt order
        self.column_keys = common.ordered_unique([self.col_fn(i) for i in results])
        self.rows, self.columns = self.populate_rows_and_columns()

In [217]:
table = CustomizedTable(
    updated_group,
    compare._colorize,
    compare._trim_significant_figures,
    compare._highlight_warnings
)
print(table.render())

[----------------------------------------------------------------------------------------------------- Resize ----------------------------------------------------------------------------------------------------]
                                                                 |  Pillow (9.0.0.post1)  |  torch (2.0.0a0+gite6bdca1) PR  |  torchvision resize  |  Speed-up: torch (2.0.0a0+gite6bdca1) PR vs torchvision resize
1 threads: --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
      3 torch.uint8 channels_last bilinear 256 -> 32 aa=True     |          38.3          |              132.5              |         367.6        |                                2.8                            
      3 torch.uint8 channels_last bilinear 256 -> 32 aa=False    |                        |              111.2              |          73.4        |    

In [196]:
common.Measurement??

Init signature:
common.Measurement(
    number_per_run: int,
    raw_times: List[float],
    task_spec: torch.utils.benchmark.utils.common.TaskSpec,
    metadata: Optional[Dict[Any, Any]] = None,
) -> None
Source:        
@dataclasses.dataclass(init=True, repr=False)
class Measurement:
    """The result of a Timer measurement.

    This class stores one or more measurements of a given statement. It is
    serializable and provides several convenience methods
    (including a detailed __repr__) for downstream consumers.
    """
    number_per_run: int
    raw_times: List[float]
    task_spec: TaskSpec
    metadata: Optional[Dict[Any, Any]] = None  # Reserved for user payloads.

    def __post_init__(self) -> None:
        self._sorted_times: Tuple[float, ...] = ()
        self._warnings: Tuple[str, ...] = ()
        self._median: float = -1.0
        self._mean: float = -1.0
        self._p25: float = -1.0
        self._p75: float = -1.0

    def __getattr__(self, name: str) -> Any:
   